# Aerosol Optical Thickness (AOT) Anomaly Detection Using Sentinel-2

In [1]:
# load creds
%load_ext dotenv
%dotenv

In [2]:
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube.core.maskset import MaskSet
from xcube.core.geom import mask_dataset_by_geometry

import os
import geopandas as gpd
import xarray as xr
import json
import IPython.display
import shapely.geometry
%matplotlib inline

In [3]:
# general parameters
run_batch = True # for batch processing
dataset = "S2L2A"
spatial_res = 0.00018 / 2 # 10m
band_names = ["AOT", "SCL"]
band_names_2020 = ["B02", "B03", "B04", "AOT", "SCL"]
t = "time"
figsize = (16, 10)

# temporal parameters of data cubes
time_period_before = "5D"
time_period_after = "1D"

In [4]:
# read areas of interest
def read_bbox(place):
    aoi = gpd.read_file(place + ".geojson").bounds.values[0]
    return aoi[0].item(), aoi[1].item(), aoi[2].item(), aoi[3].item()

place = "beijing"
bbox_beijing = read_bbox(place)
start_beijing = "2020-02-09"
end_beijing = "2020-03-15"
place = "rome"
bbox_rome = 12.35, 41.79, 12.65, 41.99
start_italy = "2020-03-09"
place = "madrid"
bbox_madrid = read_bbox(place)
start_madrid = "2020-03-14"
place = "berlin"
bbox_berlin = read_bbox(place)
place = "london"
bbox_london = bbox = read_bbox(place)
start_london = "2020-03-23"
place = "bucharest"
bbox_bucharest = bbox = read_bbox(place)
start_bucharest = "2020-03-25"
place = "paris"
bbox_paris = 2., 48.675, 2.6, 49.05
place = "brussels"
bbox_brussels = bbox = read_bbox(place)
place = "budapest"
bbox_budapest = bbox = read_bbox(place)
place = "prague"
bbox_prague = bbox = read_bbox(place)
place = "athens"
bbox_athens = 23.6, 37.935, 23.8, 38.05
place = "zurich"
bbox_zurich = bbox = read_bbox(place)

bboxes = {"beijing":bbox_beijing, "rome":bbox_rome, "madrid":bbox_madrid, 
          "berlin":bbox_berlin, "london":bbox_london, "bucharest":bbox_bucharest, 
          "paris":bbox_paris, "brussels":bbox_brussels, "budapest":bbox_budapest, 
          "prague":bbox_prague, "athens":bbox_athens, "zurich":bbox_zurich}

In [5]:
if run_batch:
    for place in bboxes:
        print("Processing: " + place)
        if not os.path.exists(place):
            is_beijing = place == "beijing"
            is_london = place == "london"
            is_italy = place in ["rome", "milan"]
            is_madrid = place == "madrid"
            is_vienna = place == "vienna"
            is_bucharest = place == "bucharest"

            bbox = bboxes.get(place)

            # cube 2017
            start = "2017-03-04"
            end = "2017-05-05"
            start = "2017-01-26" if is_beijing else start
            end = "2017-03-29" if is_beijing else end
            start = "2017-02-24" if is_italy else start
            start = "2017-03-09" if is_london else start
            start = "2017-03-01" if is_madrid else start
            start = "2017-03-11" if is_bucharest else start
            cube_config_2017 = CubeConfig(dataset_name = dataset,
                                        band_names = band_names,
                                        tile_size = [512, 512],
                                        geometry = bbox,
                                        spatial_res = spatial_res,
                                        time_range = [start, end],
                                        time_period = time_period_before)
            cube_2017 = open_cube(cube_config_2017)
            scl = MaskSet(cube_2017.SCL)
            cube_2017_masked = cube_2017.where((scl.no_data + 
                                                scl.cloud_shadows +
                                                scl.clouds_high_probability + 
                                                scl.clouds_medium_probability) == 0)
            cube_2017_masked = cube_2017_masked.drop_vars(["SCL"])

            # cube 2018
            start = "2018-03-04"
            end = "2018-05-05"
            start = "2018-01-26" if is_beijing else start
            end = "2018-03-29" if is_beijing else end
            start = "2018-02-24" if is_italy else start
            start = "2018-03-09" if is_london else start
            start = "2018-03-01" if is_madrid else start
            start = "2018-03-11" if is_bucharest else start
            cube_config_2018 = CubeConfig(dataset_name = dataset,
                                        band_names = band_names,
                                        tile_size = [512, 512],
                                        geometry = bbox,
                                        spatial_res = spatial_res,
                                        time_range = [start, end],
                                        time_period = time_period_before)
            cube_2018 = open_cube(cube_config_2018)
            scl = MaskSet(cube_2018.SCL)
            cube_2018_masked = cube_2018.where((scl.no_data + 
                                                scl.cloud_shadows +
                                                scl.clouds_high_probability + 
                                                scl.clouds_medium_probability) == 0)
            cube_2018_masked = cube_2018_masked.drop_vars(["SCL"])

            # cube 2019
            start = "2019-03-04"
            end = "2019-05-05"
            start = "2019-01-26" if is_beijing else start
            end = "2019-03-29" if is_beijing else end
            start = "2019-02-24" if is_italy else start
            start = "2019-03-09" if is_london else start
            start = "2019-03-01" if is_madrid else start
            start = "2019-03-11" if is_bucharest else start
            cube_config_2019 = CubeConfig(dataset_name = dataset,
                                         band_names = band_names,
                                         tile_size = [512, 512],
                                         geometry = bbox,
                                         spatial_res = spatial_res,
                                         time_range = [start, end],
                                         time_period = time_period_before)
            cube_2019 = open_cube(cube_config_2019)
            scl = MaskSet(cube_2019.SCL)
            cube_2019_masked = cube_2019.where((scl.no_data + 
                                                scl.cloud_shadows +
                                                scl.clouds_high_probability + 
                                                scl.clouds_medium_probability) == 0)
            cube_2019_masked = cube_2019_masked.drop_vars(["SCL"])

            # median AOT 2017, 2018 and 2019
            cube_171819 = xr.merge([cube_2017_masked, cube_2018_masked, cube_2019_masked])
            median_aot_171819 = cube_171819.AOT.median(dim = t)

            # cube 2020
            start = "2020-03-18"
            end = "2020-04-21"
            start = start_italy if is_italy else start
            start = start_beijing if is_beijing else start
            start = start_london if is_london else start
            start = start_madrid if is_madrid else start
            start = start_bucharest if is_bucharest else start
            end = end_beijing if is_beijing else end
            cube_config_2020 = CubeConfig(dataset_name = dataset,
                                         band_names = band_names_2020,
                                         tile_size = [512, 512],
                                         geometry = bbox,
                                         spatial_res = spatial_res,
                                         time_range = [start, end],
                                         time_period = "1D")
            cube_2020 = open_cube(cube_config_2020)
            scl = MaskSet(cube_2020.SCL)
            cube_2020_masked = cube_2020.where((scl.no_data + 
                                                scl.cloud_shadows +
                                                scl.clouds_high_probability + 
                                                scl.clouds_medium_probability) == 0)
            median_aot_2020 = cube_2020_masked.AOT.median(dim = t)

            # percentage difference
            diff_aot_percent = ((median_aot_2020 / median_aot_171819) * 100) - 100

            # median RGB for visualization
            median_red = cube_2020_masked.B04.median(dim = t)
            median_green = cube_2020_masked.B03.median(dim = t)
            median_blue = cube_2020_masked.B02.median(dim = t)

            # write results
            os.mkdir(place)
            suffix = "_" + place + ".nc"
            diff_aot_percent.to_netcdf(os.path.join(place, "aot_diff_percent" + suffix))
            median_aot_2020.to_netcdf(os.path.join(place, "median_aot_2020" + suffix))
            median_aot_171819.to_netcdf(os.path.join(place, "median_aot_171819" + suffix))
            median_red.to_netcdf(os.path.join(place, "B04" + suffix))
            median_green.to_netcdf(os.path.join(place, "B03" + suffix))
            median_blue.to_netcdf(os.path.join(place, "B02" + suffix))
            print("Done with: " + place)

Processing: beijing
Processing: rome
Processing: madrid
Processing: berlin
Processing: london


/opt/conda/lib/python3.7/site-packages/dask/array/core.py:3868: PerformanceWarning: Increasing number of chunks by factor of 19
  **blockwise_kwargs
/opt/conda/lib/python3.7/site-packages/dask/array/core.py:3868: PerformanceWarning: Increasing number of chunks by factor of 19
  **blockwise_kwargs
/opt/conda/lib/python3.7/site-packages/dask/array/core.py:3868: PerformanceWarning: Increasing number of chunks by factor of 19
  **blockwise_kwargs
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encounte

Done with: london
Processing: bucharest


/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


Done with: bucharest
Processing: paris


...

Done with: prague
Processing: athens


/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


Done with: athens
Processing: zurich


/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/opt/conda/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


Done with: zurich
